In [1]:
def add_final_forecast(df,cut_date):
    grouped_by_zone = df.groupby('zone')
    
    for name, group in grouped_by_zone:  
        train, test = divide_into_train_and_test(group,cut_date)
        print(name)

        if train['y'].mean(axis = 0) > 0.26:
            prophet_forecast = calculate_prophet_forecast(train, test)
            
            i = 0
            for r_idx, row in test.iterrows():
                df.loc[r_idx, 'final_forecast'] = prophet_forecast.loc[i,'yhat']
                i = i + 1
        else:
            for r_idx, row in test.iterrows():
                df.loc[r_idx, 'final_forecast'] = df.loc[r_idx, 'forecast']
    return df                

In [ ]:
def divide_into_train_and_test(df,cut_date):
    df = df[['timestamp','supply']]
    df.columns = ['ds','y']
    df = df.sort_values(by=['ds'])

    train = df[ df.ds < cut_date ]
    test = df[ df.ds >= cut_date ]    
    return train, test

In [2]:
def calculate_prophet_forecast(train, test):
    prophet = Prophet(growth="linear",holidays_prior_scale=10,yearly_seasonality=False,weekly_seasonality=False,daily_seasonality=False)
    prophet.add_country_holidays(country_name='PL'
                                ).add_seasonality(name='monthly', period=30.5, fourier_order=55
                                                 ).add_seasonality(name='weekly', period=7, fourier_order=10
                                                                  ).add_seasonality(name='daily', period=1, fourier_order=15) 
    prophet.fit(train)
    future = prophet.make_future_dataframe(periods=len(test), freq='H', include_history=False)
    forecast = prophet.predict(future)
    return forecast

In [3]:
def final_success_metric(df):
    results = []
    grouped_by_timestamp = df.groupby('timestamp')
    
    for name, group in grouped_by_timestamp:
        mean_supply = group['supply'].mean(axis = 0)
        sum_of_errors = round(abs(group['forecast'] - group['supply']),10).sum()
        single_success_metric = sum_of_errors / (mean_supply * 100)
        results.append(single_success_metric)
    return np.percentile(results, 97)

In [4]:
def success_metric(supply,forecast):
    return round(abs(forecast - supply).sum() / (supply.mean(axis = 0) * len(supply)),10)

In [4]:
def return_prophet_forecast(df, cut_date):
    grouped_by_zone = df.groupby('zone')

    for name, group in grouped_by_zone:
        group['forecast_df'] = return_single_zone_prophet_forecast(group, cut_date)
    return df

In [18]:
def return_single_zone_prophet_forecast(df,cut_date):
    data = df[['timestamp','supply']]
    data.columns = ['ds','y']
    data = data.sort_values(by=['ds'])

    train = data[ data.ds < cut_date ]
    test = data[ data.ds >= cut_date ]
    if test.shape[0] == 0:
        return float("NaN")
    else:
        prophet = Prophet(growth="linear",holidays_prior_scale=10,yearly_seasonality=False,weekly_seasonality=False,daily_seasonality=False)
        prophet.add_country_holidays(country_name='PL'
                                    ).add_seasonality(name='monthly', period=30.5, fourier_order=55
                                                     ).add_seasonality(name='weekly', period=7, fourier_order=10
                                                                      ).add_seasonality(name='daily', period=1, fourier_order=15) 
        prophet.fit(train)
        future = prophet.make_future_dataframe(periods=len(test), freq='H', include_history=False)
        forecast = prophet.predict(future)
        return forecast

In [6]:
def plot_heatmap(x,y,z,cmap='coolwarm'):
    df = pd.DataFrame.from_dict(np.array([x,y,z]).T)
    df.columns = ['X_coordinate','Y_coordinate','max_supply']
    df['max_supply'] = pd.to_numeric(df['max_supply'])

    pivotted= df.pivot('X_coordinate','Y_coordinate','max_supply')
    pivotted
    plt.figure(figsize=(12, 9))
    sns.heatmap(pivotted,annot=True,cmap=cmap)

In [7]:
def plot_comparison_heatmap(x,y,z,center=float,cmap='coolwarm'):
    df = pd.DataFrame.from_dict(np.array([x,y,z]).T)
    df.columns = ['X_coordinate','Y_coordinate','max_supply']
    df['max_supply'] = pd.to_numeric(df['max_supply'])

    pivotted= df.pivot('X_coordinate','Y_coordinate','max_supply')
    pivotted
    plt.figure(figsize=(12, 9))
    sns.heatmap(pivotted,annot=True,cmap=cmap,center=center)

In [8]:
def calculate_prophet_forecast(df_dict_zones, cut_date):
    ResultDictProphet = {elem : pd.DataFrame for elem in zone_names}

    for key, value in df_dict_zones.items():
        ResultDictProphet[key] = single_zone_prophet_forecast(value, cut_date)
    return ResultDictProphet

In [9]:
def calculate_naive_forecast(df_dict_zones, cut_date):
    ResultDictNaive = {elem : pd.DataFrame for elem in zone_names}

    for key, value in df_dict_zones.items():
        ResultDictNaive[key] = single_zone_naive_forecast(value, cut_date)
    return ResultDictNaive

In [10]:
def prepare_xyz(ResultDict):
    zone_results = pd.DataFrame.from_dict(ResultDict, orient='index')
    zone_results = zone_results.reset_index()
    first_column = zone_results.iloc[:, 0] 
    zone_results['x_coordinate'] = zone_results.apply (lambda row: row['index'][:1], axis=1)
    zone_results['y_coordinate'] = zone_results.apply (lambda row: row['index'][2:3], axis=1)
    return [zone_results['x_coordinate'],zone_results['y_coordinate'],zone_results[0]]

In [11]:
def calculate_improved_prophet_forecast(df_dict_zones, cut_date):
    ResultDictProphet = {elem : pd.DataFrame for elem in zone_names}

    for key, value in df_dict_zones.items():
        ResultDictProphet[key] = single_zone_prophet_forecast_improved(value, cut_date)
    return ResultDictProphet

In [12]:
def single_zone_naive_forecast(value, cut_date):
    data = value[['timestamp','supply']]
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data.set_index('timestamp', inplace=True)

    data_hours = data.reset_index()
    data_hours.columns = ['ds','y']
    data_hours = data_hours.sort_values(by=['ds'])
    test = data_hours[ data_hours.ds >= cut_date ]
        
    test_data = value[ value['timestamp'] >= cut_date ]
    naive_forecast = test_data['forecast']
    return success_metric(test.y.values,naive_forecast.values)

In [13]:
def single_zone_prophet_forecast(value,cut_date):
    data = value[['timestamp','supply']]
    data.columns = ['ds','y']
    data = data.sort_values(by=['ds'])

    train = data[ data.ds < cut_date ]
    test = data[ data.ds >= cut_date ]
    if test.shape[0] == 0:
        return float("NaN")
    else:
        prophet = Prophet()
        prophet.fit(train)
        future = prophet.make_future_dataframe(periods=len(test), freq='H', include_history=False)
        forecast = prophet.predict(future)
        return success_metric(test.y.values,forecast.yhat.values)

In [14]:
def single_zone_prophet_forecast_improved(value,cut_date):
    data = value[['timestamp','supply']]
    data.columns = ['ds','y']
    data = data.sort_values(by=['ds'])

    train = data[ data.ds < cut_date ]
    test = data[ data.ds >= cut_date ]
    if test.shape[0] == 0:
        return float("NaN")
    else:
        prophet = Prophet(growth="linear",holidays_prior_scale=10,yearly_seasonality=False,weekly_seasonality=False,daily_seasonality=False)
        prophet.add_country_holidays(country_name='PL'
                                    ).add_seasonality(name='monthly', period=30.5, fourier_order=55
                                                     ).add_seasonality(name='weekly', period=7, fourier_order=10
                                                                      ).add_seasonality(name='daily', period=1, fourier_order=15) 
        prophet.fit(train)
        future = prophet.make_future_dataframe(periods=len(test), freq='H', include_history=False)
        forecast = prophet.predict(future)
        return success_metric(test.y.values,forecast.yhat.values)

In [15]:
def calculate_mixed_forecast(df_dict_zones, cut_date):
    ResultDictNaive = {elem : pd.DataFrame for elem in zone_names}

    for key, value in df_dict_zones.items():
        
        ResultDictNaive[key] = single_zone_naive_forecast(value, cut_date)
    return ResultDictNaive